In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from textrec.paths import paths

In [ ]:
experiments = ['gc1', 'spec1']
frames = [pd.read_csv(paths.analyzed / f'trial_withmanual_{exp}.csv') for exp in experiments]
# Approach based on https://stackoverflow.com/a/48064892/69707
columns_ordered = []
for frame in frames:
    for col in frame.columns:
        if col not in columns_ordered:
            columns_ordered.append(col)
all_data = pd.concat(
    frames,
    keys=experiments,
    names=('experiment', '_idx'))[columns_ordered].reset_index(level=0).reset_index(drop=True)

In [ ]:
all_data['condition_mapped'] = all_data.condition.map(lambda x: {'always': 'general'}.get(x,x))

In [ ]:
all_data['condition_order_mapped'] = all_data.condition_order.map(lambda x: x.replace('always', 'general'))

In [ ]:
norecs_speed = all_data[all_data.condition == 'norecs'].groupby('participant').characters_per_sec.mean()
with_norecs_speed = pd.merge(
    all_data,
    norecs_speed.to_frame('chars_per_sec_norecs_mean'),
    left_on='participant',
    right_index=True)
with_norecs_speed

In [ ]:
with_norecs_speed['chars_per_sec_ratio_to_norecs'] = with_norecs_speed.characters_per_sec / with_norecs_speed.chars_per_sec_norecs_mean

In [ ]:
with_norecs_speed['chars_per_sec_ratio_to_norecs_log'] = np.log(with_norecs_speed.chars_per_sec_ratio_to_norecs)

In [ ]:
import scipy.stats

In [ ]:
with_norecs_speed['chars_per_sec_ratio_to_norecs_boxcox'], boxcox_lambda = scipy.stats.boxcox(with_norecs_speed.chars_per_sec_ratio_to_norecs)

In [ ]:
with_norecs_speed.to_csv(paths.analyzed / 'combined_traits.csv', index=False)